In [ ]:
def get_verb_obj_compound_from_label(label, nlp):
    """
    Given a label, return the root verb, the direct object, adjectives, and compounds.
    """
    doc = nlp(label)
    root = find_root(doc)
    if root is None:
        doc = nlp(f"to {label}")
        root = find_root(doc)
    if root is None:
        return None, None, [], []

    dobj = find_dobj(root)
    
    # Find compounds
    compounds = [token for token in doc if token.dep_ == "compound"]
    
    return root, dobj, compounds

def find_root(doc):
    for token in doc:
        if token.dep_ == "ROOT" and token.pos_ == "VERB":
            return token
    return None

def find_dobj(token):
    for child in token.children:
        if child.dep_ in ("dobj", "pobj"):
            return child
        elif child.dep_ in ("prep", "adp"):
            return find_dobj(child)
    return None

In [ ]:
def generate_pattern(label):
    root, obj, compounds = get_verb_obj_compound_from_label(label, nlp)
    pattern = []
    
    # Aggiungi il verbo radice se trovato
    if root:
        pattern.append({"LEMMA": root.lemma_, "POS": "VERB"})
    
    # Aggiungi i compound al pattern
    for compound in compounds:
        pattern.append({"LOWER": compound.text.lower(), "DEP": "compound"})
    
    # Aggiungi l'oggetto diretto se trovato
    if obj:
        pattern.append({"LEMMA": obj.lemma_, "POS": "NOUN"})
    
    # Se non è stato generato alcun pattern, usa l'etichetta originale
    if not pattern:
        pattern = [{"LOWER": label.lower()}]
    
    return pattern

In [ ]:
import spacy
from spacy.matcher import Matcher

labels = [
          'customer satisfaction guarantee',
          'ensure customer satisfaction',
          'guarantee customer satisfaction',
          'guaranteeing customer satisfaction',
          'promise customer satisfaction',
          'provide customer satisfaction',
          'to guarantee customer satisfaction']

# Crea un matcher
matcher = Matcher(nlp.vocab)

# Genera e stampa i pattern per ogni etichetta
for label in labels:
    pattern = generate_pattern(label)
    print(f"Label: {label}")
    print(f"Pattern: {pattern}")
    print()

    # Aggiungi il pattern al matcher
    matcher.add(label, [pattern])

# Esempio di utilizzo del matcher
def find_matches(text):
    doc = nlp(text)
    matches = matcher(doc)
    for match_id, start, end in matches:
        string_id = nlp.vocab.strings[match_id]
        span = doc[start:end]
        print(f"Matched '{string_id}': {span.text}")

# Test del matcher
test_text = "Results-driven Customer Experience Manager with 5+ years of expertise in ensuring customer satisfaction. Proven track record of implementing effective strategies to guarantee customer satisfaction across all touchpoints. Developed a customer satisfaction guarantee program that increased retention rates by 15%. Skilled in training teams to provide exceptional service and maintain high satisfaction levels. Consistently exceeded targets in promising and delivering customer satisfaction, resulting in a 95% positive feedback rate. Seeking a challenging role to further enhance customer experience and drive business growth through unwavering commitment to customer satisfaction."
print("Testing matcher with:", test_text)
find_matches(test_text)

```
Label: customer satisfaction guarantee
Pattern: [{'LEMMA': 'guarantee', 'POS': 'VERB'}, {'LOWER': 'customer', 'DEP': 'compound'}, {'LOWER': 'satisfaction', 'DEP': 'compound'}]

Label: ensure customer satisfaction
Pattern: [{'LEMMA': 'ensure', 'POS': 'VERB'}, {'LOWER': 'customer', 'DEP': 'compound'}, {'LEMMA': 'satisfaction', 'POS': 'NOUN'}]

Label: guarantee customer satisfaction
Pattern: [{'LEMMA': 'guarantee', 'POS': 'VERB'}, {'LOWER': 'customer', 'DEP': 'compound'}, {'LEMMA': 'satisfaction', 'POS': 'NOUN'}]

Label: guaranteeing customer satisfaction
Pattern: [{'LEMMA': 'guarantee', 'POS': 'VERB'}, {'LOWER': 'customer', 'DEP': 'compound'}, {'LEMMA': 'satisfaction', 'POS': 'NOUN'}]

Label: promise customer satisfaction
Pattern: [{'LEMMA': 'promise', 'POS': 'VERB'}, {'LOWER': 'customer', 'DEP': 'compound'}, {'LEMMA': 'satisfaction', 'POS': 'NOUN'}]

Label: provide customer satisfaction
Pattern: [{'LEMMA': 'provide', 'POS': 'VERB'}, {'LOWER': 'customer', 'DEP': 'compound'}, {'LEMMA': 'satisfaction', 'POS': 'NOUN'}]

Label: to guarantee customer satisfaction
Pattern: [{'LEMMA': 'guarantee', 'POS': 'VERB'}, {'LOWER': 'customer', 'DEP': 'compound'}, {'LEMMA': 'satisfaction', 'POS': 'NOUN'}]

Testing matcher with: Results-driven Customer Experience Manager with 5+ years of expertise in ensuring customer satisfaction. Proven track record of implementing effective strategies to guarantee customer satisfaction across all touchpoints. Developed a customer satisfaction guarantee program that increased retention rates by 15%. Skilled in training teams to provide exceptional service and maintain high satisfaction levels. Consistently exceeded targets in promising and delivering customer satisfaction, resulting in a 95% positive feedback rate. Seeking a challenging role to further enhance customer experience and drive business growth through unwavering commitment to customer satisfaction.
Matched 'ensure customer satisfaction': ensuring customer satisfaction
Matched 'guarantee customer satisfaction': guarantee customer satisfaction
Matched 'guaranteeing customer satisfaction': guarantee customer satisfaction
```
